# Text Classification Models Comparison

In this tutorial, you'll learn to make text classification on a dataset using two different models, to upload models predictions in your Argilla workspace and to compare models by computing F1 score for each model. It will walk you through this steps:
- 💾 Load the dataset you want to use.
- 💻 Compute predictions with zero-shot classification model.
- 🔄 Convert model output to Argilla format and upload it to Argilla workspace.
- 💻 Compute predictions with few-shots classification model.
- 🧪 Compare models predictions with F1 score


## Introduction
When working on Text Classification, you may want to compare two models to decide which one to use.
For this we compute the F1 score on train models using their annotations as the true text class.
The F1 score can be interpreted as a harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. (more info in this [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html))

*Argilla* allows you to deploy and monitor any model you like, but in this tutorial we will focus on the two most common frameworks in the NLP space:[transformers](https://huggingface.co/docs/transformers) and [classy-classification](https://github.com/Pandora-Intelligence/classy-classification). Let's get started!

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:


**Deploy Argilla on Hugging Face Spaces**: If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).


**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](../../getting_started/quickstart.ipynb). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

## Setup

To complete this tutorial, you will need to install the Argilla client and a few third party libraries using `pip`:

In [ ]:
%pip install transformers argilla datasets classy_classification torch tqdm -qqqqqq

The imports needed:

In [2]:
import argilla as rg
from datasets import load_dataset
from transformers import pipeline
from argilla.metrics.text_classification import f1
import pandas as pd
from classy_classification import ClassyClassifier

If you are running Argilla using the Docker quickstart image or Hugging Face Spaces, you need to init the Argilla client with the `URL` and `API_KEY`:

In [3]:
rg.init(
    api_url="URL",
    api_key="API_KEY",
)

For this tutorial the HugginFace [ag_news](https://https://huggingface.co/datasets/ag_news) dataset is chosen:

In [ ]:
news_dataset = load_dataset("ag_news", split="test")

This dataset is composed of two columns, one is the text of the news article and the other one is the label associated to this text article:

*For this tutorial, we will consider the label as the annotation of the text*



We transform our dataset in order to create an argilla TextClassificationDataset:

In [ ]:
int_to_label = {
    0:"World",
    1:"Sports",
    2:"Business",
    3:"Sci/Tech",
}

news_dataset = news_dataset.map(lambda row: {"prediction": [{"label":int_to_label[row["label"]], "score":1}]})

In [ ]:
ds_record = rg.read_datasets(dataset=news_dataset, task="TextClassification")

### Make Zero Shot Text Classification predictions using transformers

On HugginFace we choose the model `cross-encoder/nli-distilroberta-base` that is trained to perform zero-shot classification.
We create a pipeline with this model and then perform prediction.

*note: `device=0` in pipeline() permits to use GPU if you do not have a GPU available delete this parameter*




In [ ]:
labels =["Sports", "Sci/Tech", "Business", "World"]

pipe = pipeline("zero-shot-classification", model='cross-encoder/nli-distilroberta-base', device=0)
result = []
with pipe.device_placement():
  result = pipe(
    [data.text for data in ds_record],
    candidate_labels=labels,
  )

Now that predictions are successfully made with the zero-shot model we can transform it to a list of argilla TextClassificationRecord and upload it to our argilla client

In [ ]:
zero_shot_news_dataset = [
    rg.TextClassificationRecord(
        text=res["sequence"],
        prediction=list(zip(res['labels'],res['scores'])),
        annotation=record.prediction[0][0],
    ) for res, record in zip(result, ds_record)
]

rg.log(name="zero_shot_news_dataset", records=zero_shot_news_dataset)

You can acces the zero_shot_news_dataset in the Argilla UI:
![zero-shot-dataset](../../_static/tutorials/model-comparison-for-text-classification/zero-shot-dataset.png)

Finally, we measure the performance of our model using the argilla f1 function that computes F1 score:

In [23]:
f1_zero_shot = f1("zero_shot_news_dataset")
f1_zero_shot.visualize()

### Make Few Shots Text Classification using classy_classification
Few shots classification is computed using a pre-trained model to generalize over new categories of data using only a few labeled samples per class.

Here we are training the model using a sample of 7 data.

We need to create the dict of annotated texts per label:

In [ ]:
# Get some annotated examples per class
n_samples_per_class = 7
data = {"World": [], "Sports": [], "Sci/Tech": [], "Business": []}
train_indices=[]
while not all([len(value)== n_samples_per_class for key,value in data.items()]):
    for idx, rec in enumerate(ds_record):
        if len(data[rec.prediction[0][0]]) < n_samples_per_class:
            data[rec.prediction[0][0]].append(rec.text)
            train_indices.append(idx)

test_ds = [rec for idx, rec in enumerate(ds_record) if idx not in train_indices] # We delete the records used for the few-shots training of ds_record

We train our model with the few annotated data calculated in the cell above. We are using the model `all-MiniLM-L6-v2` 

In [ ]:
# train a few-shot classifier
classifier = ClassyClassifier(data=data, model="all-MiniLM-L6-v2")

With this trained classifier we can perform prediction:

In [ ]:
few_shot_results = [classifier(record.text) for record in test_ds]

Now that predictions are successfully made with the few-shots model we can transform it to a list of argilla TextClassificationRecord and upload it to our argilla client

In [ ]:
few_shots_news_dataset = [
    rg.TextClassificationRecord(
        text=record.text,
        prediction=list(res.items()),
        annotation=record.prediction[0][0],
    ) for record, res in zip(ds_record, few_shot_results)
]

rg.log(name="few_shots_news_dataset", records=few_shots_news_dataset)

You can acces the few_shots_news_dataset in the Argilla UI:
![few-shots-dataset](../../_static/tutorials/model-comparison-for-text-classification/few-shots-dataset.png)

Finally, we measure the performance of our model using the argilla f1 function that computes f-score:

In [22]:
f1_few_shots = f1("few_shots_news_dataset")
f1_few_shots.visualize()

Now that we have computed the F1 score for each model with can create a comparison table:



In [20]:
f_score = list(f1_few_shots.data.keys())
f1_few_shots_values = list(f1_few_shots.data.values())
f1_zero_shot_values = list(f1_zero_shot.data.values())

unnecessary_labels = ["Sports_recall", "World_recall", ""]

df_results = pd.DataFrame({"f_score": f_score, "zero-shot classification": f1_zero_shot_values, "few-shots classification": f1_few_shots_values})


In [21]:
df_results

,f_score,zero-shot classification,few-shots classification
0,precision_macro,0.517408,0.155022
1,recall_macro,0.528304,0.166217
2,f1_macro,0.513818,0.156829
3,precision_micro,0.529538,0.164430
4,recall_micro,0.529538,0.164430
5,f1_micro,0.529538,0.164430
6,Sports_precision,0.441704,0.140809
7,Sports_recall,0.563833,0.120752
8,Sports_f1,0.495352,0.130012
9,Sports_support,5812.000000,4149.000000


*Results interpretation:* Without any hesitation the zero-shot classification model is the more effictive than the few-shots one. The F1 score for each class is better.

The best predicted class for zero-shot classifier is **Sci/Tech**. It is the class **World** for the few-shots classifier.